In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_unit 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 
AS

WITH prp AS (
    SELECT 
        LTRIM('0', rpa_mean) AS material, 
        rpa_disqu AS mat_unit, 
        ARRAY_JOIN(COLLECT_LIST(eanupc), ";") AS barcode_full
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_prpa_mean a 
    WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_prpa_mean)
    GROUP BY material, mat_unit
),
temp AS (
    SELECT 
        a.material product_id,
        a.mat_unit unit,
        a.eanupc barcode,
        prp.barcode_full,
        a.denomintr,
        a.numerator,
        a.gross_wt gross_weight,
        a.unit_of_wt,
        a.height,
        a.len,
        a.width,        
        a.unit_dim,
        a.volume,
        a.volumeunit,
        b.base_uom,
        ROW_NUMBER() OVER (PARTITION BY a.material, a.mat_unit ORDER BY proc_date DESC) rn
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_pmat_unit a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.material = b.product_id
    LEFT JOIN prp ON a.material = prp.material AND a.mat_unit = prp.mat_unit
    WHERE a.end_date = '2400-01-01'
)
SELECT * EXCEPT(rn) 
FROM temp
WHERE rn = 1 AND product_id <> ''
 """)